In [1]:
import pandas as pd
import numpy as np
from google.cloud import bigquery
%load_ext google.cloud.bigquery

# 1. Preparing the Matching tables

As we are using multiple sources of data, matching the data from one source to another was really importan for this project. Thus, we prepared two matching table for the future users who wants to use our dataset.

## 1.1. Matching Original Application Numbers to Application IDs (PATSTAT)

This table will match the application numbers which can be found in the publications of each filing to its application ID which can be found in the PATSTAT dataset.

**Source:** For creating this table we are use "1_matching_applnNr_applnId" table that is created in the "data_preparation" step. Please refer to the relevant notebook in "data_preparation" directory, for more information

In [3]:
client = bigquery.Client()
# Creating Job Config
job_config = bigquery.QueryJobConfig()
#job_config.dry_run = True
job_config.use_query_cache = False
# Set configuration.query.writeDisposition
job_config.write_disposition = 'WRITE_TRUNCATE'

# Set the destination table
dataset_id = 'final_dataset'
table_id = '08_matching_applnNrOrig_applnId'
table_ref = client.dataset(dataset_id).table(table_id)
job_config.destination = table_ref

query= """
SELECT 
    appln_id,
    appln_nr_orig
FROM `usptobias.data_preparation.1_matching_applnNr_applnId`
"""

# Defining the query
query_job = client.query(query, location='US', job_config=job_config)

query_job.result()

## 1.2. Matching Application PAIR numbers to the Original Application Numbers

This table will match the special application numbers found in the USPTO PAIR dataset to the original application numbers found in the publication of patents.

**Source:** This table is taken directly from the USPTO PAIR dataset.

In [4]:
client = bigquery.Client()
# Creating Job Config
job_config = bigquery.QueryJobConfig()
#job_config.dry_run = True
job_config.use_query_cache = False
# Set configuration.query.writeDisposition
job_config.write_disposition = 'WRITE_TRUNCATE'

# Set the destination table
dataset_id = 'final_dataset'
table_id = '09_matching_applnNrPAIR_applnNrOrig'
table_ref = client.dataset(dataset_id).table(table_id)
job_config.destination = table_ref

query= """
SELECT 
    application_number_pair AS appln_nr_PAIR, 
    application_number AS appln_nr_orig
FROM `patents-public-data.uspto_oce_pair.match`
"""

# Defining the query
query_job = client.query(query, location='US', job_config=job_config)

query_job.result()

# 2. Creating Attorney Table

This table contains the attorney names and their unique IDs for each application ID.

**Source:** This table is created in the 'data_preparation' step. Please refer to `5_attorney_tables` notebook in the "./data_preparation" directory, for more information.

In [7]:
client = bigquery.Client()
# Creating Job Config
job_config = bigquery.QueryJobConfig()
#job_config.dry_run = True
job_config.use_query_cache = False
# Set configuration.query.writeDisposition
job_config.write_disposition = 'WRITE_TRUNCATE'

# Set the destination table
dataset_id = 'final_dataset'
table_id = '01_applnId_attorney'
table_ref = client.dataset(dataset_id).table(table_id)
job_config.destination = table_ref

query= """
WITH ApplnLawyer_table AS(
    SELECT * EXCEPT(rank)
    FROM(
        SELECT 
            *,
            ROW_NUMBER() OVER (PARTITION BY appln_nr) AS rank
        FROM `usptobias.data_preparation.5_appln_attorney`
    )
    WHERE rank=1
)

SELECT 
    appln_id,
    attorney,
    attorney_id,
    attorney_region_code,
    attorney_country_code
FROM ApplnLawyer_table AS a
INNER JOIN `usptobias.final_dataset.09_matching_applnNrPAIR_applnNrOrig` AS b ON b.appln_nr_PAIR = a.appln_nr
LEFT JOIN `usptobias.final_dataset.08_matching_applnNrOrig_applnId` AS c ON c.appln_nr_orig = b.appln_nr_orig
"""

# Defining the query
query_job = client.query(query, location='US', job_config=job_config)

query_job.result()

# 3. Creating A Table Containing Inventors, Applicants, and Claims information

**Source:** This table is created in the 'data_preparation' step. Please refer to `4_invtAppltClaim_appln` notebook in the "./data_preparation" directory, for more information.

In [8]:
client = bigquery.Client()
# Creating Job Config
job_config = bigquery.QueryJobConfig()
#job_config.dry_run = True
job_config.use_query_cache = False
# Set configuration.query.writeDisposition
job_config.write_disposition = 'WRITE_TRUNCATE'

# Set the destination table
dataset_id = 'final_dataset'
table_id = '02_applnId_publn'
table_ref = client.dataset(dataset_id).table(table_id)
job_config.destination = table_ref

query= """
SELECT 
    appln_id, publn_auth, publn_kind, publn_claims_earliest, publn_claims_grant, 
    publn_date_earliest, publn_date_grant, nb_applt, nb_invt
FROM `usptobias.data_preparation.4_invtAppltClaim_appln`
"""

# Defining the query
query_job = client.query(query, location='US', job_config=job_config)

query_job.result()

# 4. Creating Inventors Table Using USPTO PAIR

This table contains inventors information that have been extracted from the USPTO PAIR dataset.

**Source:** For preparing this table we use `3_appln_uspto` table that is created in the data preparation step (more information: './data_preparation/3_appln_uspto' notebook) and `all_inventors` table form USPTO PAIR dataset.

In [13]:
client = bigquery.Client()
# Creating Job Config
job_config = bigquery.QueryJobConfig()
#job_config.dry_run = True
job_config.use_query_cache = False
# Set configuration.query.writeDisposition
job_config.write_disposition = 'WRITE_TRUNCATE'

# Set the destination table
dataset_id = 'final_dataset'
table_id = '03_appln_inventors_uspto'
table_ref = client.dataset(dataset_id).table(table_id)
job_config.destination = table_ref

query= """
SELECT 
    appln_id,
    UPPER(inventor_name_first) AS invt_name_first,
    UPPER(inventor_name_middle) AS invt_name_middle,
    UPPER(inventor_name_last) AS invt_name_last,
    inventor_rank AS invt_seq_nr,
    (CASE WHEN inventor_country_code='US' THEN 1 ELSE 0 END) AS is_US_resident,
    inventor_country_code AS invt_country_code
FROM `patents-public-data.uspto_oce_pair.all_inventors` AS a
INNER JOIN `usptobias.final_dataset.09_matching_applnNrPAIR_applnNrOrig` AS b ON b.appln_nr_PAIR=a.application_number
LEFT JOIN `usptobias.final_dataset.08_matching_applnNrOrig_applnId` AS c ON c.appln_nr_orig = b.appln_nr_orig
INNER JOIN `usptobias.data_preparation.3_appln_uspto` AS d USING(appln_id)
"""

# Defining the query
query_job = client.query(query, location='US', job_config=job_config)

query_job.result()

# 5. Creating Examiners Table

This table contains examiners' names with their unique ID.

**Source:** For creating this table, we are using the `3_appln_uspto` tabel that is created in the "data_prepration" step.

In [14]:
client = bigquery.Client()
# Creating Job Config
job_config = bigquery.QueryJobConfig()
#job_config.dry_run = True
job_config.use_query_cache = False
# Set configuration.query.writeDisposition
job_config.write_disposition = 'WRITE_TRUNCATE'

# Set the destination table
dataset_id = 'final_dataset'
table_id = '04_examiners_uspto'
table_ref = client.dataset(dataset_id).table(table_id)
job_config.destination = table_ref

query= """
SELECT
    examiner_id,
    examiner_name.first AS examiner_name_first,
    examiner_name.middle AS examiner_name_middle,
    examiner_name.last AS examiner_name_last,
    examiner_art_unit
FROM(
    SELECT 
        examiner_id,
        ANY_VALUE(examiner_name) AS examiner_name, 
        MAX(examiner_art_unit) AS examiner_art_unit
    FROM `usptobias.data_preparation.3_appln_uspto`
    GROUP BY examiner_id
)
"""

# Defining the query
query_job = client.query(query, location='US', job_config=job_config)

query_job.result()

# 6. Creating Application info Table Using USPTO PAIR Dataset

In this table, we many different information for each application that will be used in the analyzing step. This information includes: filing date, grant_date, small_entity, examiner ID, examiner art unit, number of office actions, number of transactions, and many more.

**Source:** For creating this table, we use the `3_appln_uspto` and `7_officeAction_category` tabels that are created in the "data_prepration" step.

In [15]:
client = bigquery.Client()
# Creating Job Config
job_config = bigquery.QueryJobConfig()
#job_config.dry_run = True
job_config.use_query_cache = False
# Set configuration.query.writeDisposition
job_config.write_disposition = 'WRITE_TRUNCATE'

# Set the destination table
dataset_id = 'final_dataset'
table_id = '05_applnInfo_uspto'
table_ref = client.dataset(dataset_id).table(table_id)
job_config.destination = table_ref

query= """
WITH transactions_t AS(
    SELECT 
        appln_id, 
        SUM(CASE WHEN transactions.event_code='CTNF' OR transactions.event_code='CTFR' THEN 1 ELSE 0 END) as nb_rejection,
        COUNT(transactions.event_code) AS nb_transaction
    FROM `usptobias.data_preparation.3_appln_uspto` a, a.transactions
    GROUP BY appln_id
), t1 AS(
    SELECT transaction, category 
    FROM `usptobias.data_preparation.7_officeAction_category`
), t2 AS(
    SELECT 
        appln_id, 
        transactions
    FROM `usptobias.data_preparation.3_appln_uspto` a, a.transactions
), t3 AS(
SELECT appln_id, transaction AS event_code, category AS transaction_cat
FROM t2
LEFT JOIN t1 ON t2.transactions.event_code=t1.transaction
)

SELECT 
    appln_id AS appln_id,
    ANY_VALUE(appln_type) AS appln_type,
    ANY_VALUE(PARSE_DATE("%F",filing_date)) AS filing_date,
    ANY_VALUE(PARSE_DATE("%F", grant_date)) AS grant_date,
    ANY_VALUE(PARSE_DATE("%F", issue_date)) AS issue_date,
    ANY_VALUE(PARSE_DATE("%F", abandon_date)) AS abandon_date,
    ANY_VALUE(small_entity) AS small_entity,
    ANY_VALUE(disposal_type) AS disposal_type,
    ANY_VALUE(examiner_id) AS examiner_id,
    ANY_VALUE(examiner_art_unit) AS examiner_art_unit,
    MAX(ARRAY_LENGTH(actions)) AS nb_office_actions,
    MAX(nb_rejection) AS nb_rejection,
    MAX(nb_transaction) AS nb_transaction,
    COUNT(CASE WHEN transaction_cat='EX' THEN 1 END) AS nb_transaction_ex,
    COUNT(CASE WHEN transaction_cat='AA' THEN 1 END) AS nb_transaction_aa,
    ANY_VALUE(patent_nr) AS patent_nr,
    ANY_VALUE(status_code) AS status_code,
    MAX(EXTRACT(DATE FROM PARSE_TIMESTAMP("%d%b%Y%t%X", status_date))) AS status_date
FROM `usptobias.data_preparation.3_appln_uspto`
LEFT JOIN transactions_t USING(appln_id)
LEFT JOIN t3 USING(appln_id)
GROUP BY appln_id

"""

# Defining the query
query_job = client.query(query, location='US', job_config=job_config)

query_job.result()

# 7. Creating Custom Family ID table

This table contains the application IDs and the assigned custom family ID.

**Source:** For creating this table we use `Cutom_familyId` table from the "computing_familyID" step. Please refer to `./computing_familyID/Custom_FamilyID` for more information.

In [19]:
client = bigquery.Client()
# Creating Job Config
job_config = bigquery.QueryJobConfig()
#job_config.dry_run = True
job_config.use_query_cache = False
# Set configuration.query.writeDisposition
job_config.write_disposition = 'WRITE_TRUNCATE'

# Set the destination table
dataset_id = 'final_dataset'
table_id = '06_family_customDef'
table_ref = client.dataset(dataset_id).table(table_id)
job_config.destination = table_ref

query= """
SELECT 
    *
FROM `usptobias.custom_family.family_customDef`
"""

# Defining the query
query_job = client.query(query, location='US', job_config=job_config)

query_job.result()

# 8. Creating Twin Applications Table

This table contains twin patents, where one of the applications is always from the U.S. jurisdiction.

**Source:** For creating this table we use `twin_appln` table from "computing_family" step. Please refer to `./computing_familyID/Custom_FamilyID` notebook, for more information.

In [20]:
client = bigquery.Client()
# Creating Job Config
job_config = bigquery.QueryJobConfig()
#job_config.dry_run = True
job_config.use_query_cache = False
# Set configuration.query.writeDisposition
job_config.write_disposition = 'WRITE_TRUNCATE'

# Set the destination table
dataset_id = 'final_dataset'
table_id = '07_twin_appln'
table_ref = client.dataset(dataset_id).table(table_id)
job_config.destination = table_ref

query= """
SELECT 
    *
FROM `usptobias.custom_family.twin_appln`
WHERE appln_auth_2 IN('EP', 'JP', 'CN', 'KR','DE', 'CA', 'AU', 'TW')
"""

# Defining the query
query_job = client.query(query, location='US', job_config=job_config)

query_job.result()

# 9. Creating Pure Assignee Table

This table contains pure assigne information for each application. By pure assignee, we mean assignees that are not listed as inventors.

**Source:** For creating this table, we have used `TLS201_APPLN`, `TLS207_PERS_APPLN`, `TLS906_PERSON` table from the PATSTAT dataset.

In [21]:
client = bigquery.Client()
# Creating Job Config
job_config = bigquery.QueryJobConfig()
#job_config.dry_run = True
job_config.use_query_cache = False
# Set configuration.query.writeDisposition
job_config.write_disposition = 'WRITE_TRUNCATE'

# Set the destination table
dataset_id = 'final_dataset'
table_id = '10_assignee_appln' # 2_pure_assignee_appln
table_ref = client.dataset(dataset_id).table(table_id)
job_config.destination = table_ref

query= """
WITH t1 AS(
    SELECT appln_id
    FROM `usptobias.patstat.tls201`
    WHERE appln_auth='US'
), t2 AS(
        SELECT *
        FROM `usptobias.patstat.tls207`
        INNER JOIN t1 USING(appln_id)
        WHERE applt_seq_nr<>0 AND invt_seq_nr=0
)

SELECT 
    appln_id,
    person_id AS assignee_id,
    b.han_name AS assignee_han_name,
    b.psn_sector AS assignee_sector,
    applt_seq_nr,
    person_ctry_code AS assignee_country_code
FROM t2
LEFT JOIN `usptobias.patstat.tls906` b USING(person_id)
"""

# Defining the query
query_job = client.query(query, location='US', job_config=job_config)

query_job.result()

# 10. Creating Pseudo Fixed Effect Tables

## 10.1. Attorney Pseudo Fixed Effect

This table contains attorney pseudo fixed effects on assignees.

**Source:** For creating this table we used `01_applnId_attorney`, `10_assignee_appln` from "final_dataset" and `TLS201_APPLN` table from PATSTAT.

In [22]:
client = bigquery.Client()
# Creating Job Config
job_config = bigquery.QueryJobConfig()
#job_config.dry_run = True
job_config.use_query_cache = False
# Set configuration.query.writeDisposition
job_config.write_disposition = 'WRITE_TRUNCATE'

# Set the destination table
dataset_id = 'final_dataset'
table_id = '11_attorneyPFE_on_assignee'
table_ref = client.dataset(dataset_id).table(table_id)
job_config.destination = table_ref

query= """
WITH assignee AS(
    SELECT appln_id, assignee_id
    FROM `usptobias.final_dataset.10_assignee_appln`
), attorney AS(
    SELECT * EXCEPT(row_attorney)
    FROM(
        SELECT appln_id, attorney_id, ROW_NUMBER() OVER(PARTITION BY appln_id) row_attorney
        FROM `usptobias.final_dataset.01_applnId_attorney`
    )
    WHERE row_attorney=1
), attorney_assignee AS(
    SELECT
        appln_id,
        assignee_id,
        attorney_id,
        CAST(CAST(b.granted AS INT64) AS FLOAT64) granted
    FROM assignee
    INNER JOIN attorney USING(appln_id)
    INNER JOIN (SELECT appln_id, granted FROM `usptobias.patstat.tls201` WHERE appln_auth='US') b USING(appln_id)
), attorney_granted AS(
    SELECT
        attorney_id,
        ARRAY_AGG(STRUCT(assignee_id, granted)) assignee_arr
    FROM attorney_assignee
    GROUP BY attorney_id
), attorney_fixed AS(
    SELECT 
        assignee_id,
        attorney_id,
        ARRAY(SELECT c.granted FROM b.assignee_arr AS c WHERE a.assignee_id<>c.assignee_id) grant_arr
    FROM attorney_assignee AS a
    INNER JOIN attorney_granted AS b USING(attorney_id)
)


SELECT
    assignee_id,
    attorney_id,
    AVG(grant_arr) AS grant_rate
FROM
attorney_fixed AS a, a.grant_arr
GROUP BY assignee_id, attorney_id
"""

# Defining the query
query_job = client.query(query, location='US', job_config=job_config)

query_job.result()

## 10.2. Art Unit Pseudo Fixed Effect

This table contains art unit pseudo fixed effects on assignees.

**Source:** For creating this table we used `05_applnInfo_uspto`, `10_assignee_appln` from "final_dataset" and `TLS201_APPLN` table from PATSTAT.

In [23]:
client = bigquery.Client()
# Creating Job Config
job_config = bigquery.QueryJobConfig()
#job_config.dry_run = True
job_config.use_query_cache = False
# Set configuration.query.writeDisposition
job_config.write_disposition = 'WRITE_TRUNCATE'

# Set the destination table
dataset_id = 'final_dataset'
table_id = '12_artunitPFE_on_assignee'
table_ref = client.dataset(dataset_id).table(table_id)
job_config.destination = table_ref

query= """
WITH assignee AS(
    SELECT appln_id, assignee_id
    FROM `usptobias.final_dataset.10_assignee_appln`
), artunit_t AS(
    SELECT * EXCEPT(row_artunit)
    FROM(
        SELECT appln_id, examiner_art_unit, ROW_NUMBER() OVER(PARTITION BY appln_id) row_artunit
        FROM `usptobias.final_dataset.05_applnInfo_uspto`
    )
    WHERE row_artunit=1
), artunit_assignee AS(
    SELECT
        appln_id,
        assignee_id,
        examiner_art_unit,
        CAST(CAST(b.granted AS INT64) AS FLOAT64) granted
    FROM assignee
    INNER JOIN artunit_t USING(appln_id)
    INNER JOIN (SELECT appln_id, granted FROM `usptobias.patstat.tls201` WHERE appln_auth='US') b USING(appln_id)
), artunit_grant AS(
    SELECT
        examiner_art_unit,
        ARRAY_AGG(STRUCT(assignee_id, granted)) assignee_arr
    FROM artunit_assignee
    GROUP BY examiner_art_unit
), artunit_fixed AS(
    SELECT 
        assignee_id,
        examiner_art_unit,
        ARRAY(SELECT c.granted FROM b.assignee_arr AS c WHERE a.assignee_id<>c.assignee_id) grant_arr
    FROM artunit_assignee AS a
    INNER JOIN artunit_grant AS b USING(examiner_art_unit)
)


SELECT
    assignee_id,
    examiner_art_unit,
    AVG(grant_arr) AS grant_rate
FROM artunit_fixed AS a, a.grant_arr
GROUP BY assignee_id, examiner_art_unit
"""

# Defining the query
query_job = client.query(query, location='US', job_config=job_config)

query_job.result()

## 10.3. Examiner Pseudo Fixed Effect

This table contains examiner pseudo fixed effects on assignees.

**Source:** For creating this table we used `05_applnInfo_uspto`, `10_assignee_appln` from "final_dataset" and `TLS201_APPLN` table from PATSTAT.

In [24]:
client = bigquery.Client()
# Creating Job Config
job_config = bigquery.QueryJobConfig()
#job_config.dry_run = True
job_config.use_query_cache = False
# Set configuration.query.writeDisposition
job_config.write_disposition = 'WRITE_TRUNCATE'

# Set the destination table
dataset_id = 'final_dataset'
table_id = '13_examinerPFE_on_assignee'
table_ref = client.dataset(dataset_id).table(table_id)
job_config.destination = table_ref

query= """
WITH assignee AS(
    SELECT appln_id, assignee_id
    FROM `usptobias.final_dataset.10_assignee_appln`
), examiners AS(
    SELECT * EXCEPT(row_examiner)
    FROM(
        SELECT appln_id, examiner_id, ROW_NUMBER() OVER(PARTITION BY appln_id) row_examiner
        FROM `usptobias.final_dataset.05_applnInfo_uspto`
    )
    WHERE row_examiner=1
), examiner_assignee AS(
    SELECT
        appln_id,
        assignee_id,
        examiner_id,
        CAST(CAST(b.granted AS INT64) AS FLOAT64) granted
    FROM assignee
    INNER JOIN examiners USING(appln_id)
    INNER JOIN (SELECT appln_id, granted FROM `usptobias.patstat.tls201` WHERE appln_auth='US') b USING(appln_id)
), examiner_grant AS(
    SELECT
        examiner_id,
        ARRAY_AGG(STRUCT(assignee_id, granted)) assignee_arr
    FROM examiner_assignee
    GROUP BY examiner_id
), examiner_fixed AS(
    SELECT 
        assignee_id,
        examiner_id,
        ARRAY(SELECT c.granted FROM b.assignee_arr AS c WHERE a.assignee_id<>c.assignee_id) grant_arr
    FROM examiner_assignee AS a
    INNER JOIN examiner_grant AS b USING(examiner_id)
)


SELECT
    assignee_id,
    examiner_id,
    AVG(grant_arr) AS grant_rate
FROM examiner_fixed AS a, a.grant_arr
GROUP BY assignee_id, examiner_id
"""

# Defining the query
query_job = client.query(query, location='US', job_config=job_config)

query_job.result()

# 11. Name's Ethnicity Table

`14_name_ethnicity` table contains information on the predicted ethnicity of (first name, last name) pairs using [NamePrism API](http://www.name-prism.com/api).

**Source:** For creating this table we used predicted ethnicity [NamePrism API](http://www.name-prism.com/api). More information is available in the `Name_Ethnicty_Prediction` notebook.

# 12. Name's Origin and Gender Tables

`15_name_gender` and `16_name_origin` tables contain information on the predicted country of origin and predicted gender of (first name, last name, country of residence) triples using [NamSor API](https://www.namsor.com/).

**Source:** For creating this table we used the predicted country of origin and gender from [NamSor API](https://www.namsor.com/). More information is available in the `Name_Origin_Prediction` and `Name_Gender_Prediction` notebooks.

# 13. Creating Country of Origin and Residence Table

This table contains information on the country of origin, country of residence, and ethnicity information of examiners and inventors for each application.  
For this purpose, we first need to find the origin, gender, and ethnicty of inventors and examiners using three `14_name_ethnicity`, `15_name_gender`, and `16_name_origin` table. Tables `17_invtOrig_appln` and `17_examOrig_appln` are built for this purpose.  

**Source:** For creating this table, we use `14_name_ethnicity`, `15_name_gender`, and `16_name_origin` tables for the prediction data and `03_appln_inventor_uspto` / `04_examiners_uspto` for the inventors/examiners data.

In [27]:
client = bigquery.Client()
# Creating Job Config
job_config = bigquery.QueryJobConfig()
#job_config.dry_run = True
job_config.use_query_cache = False
# Set configuration.query.writeDisposition
job_config.write_disposition = 'WRITE_TRUNCATE'

# Set the destination table
dataset_id = 'final_dataset'
table_id = '17_invtOrig_appln'
table_ref = client.dataset(dataset_id).table(table_id)
job_config.destination = table_ref

query= """
WITH invnt_table AS(
    SELECT 
        appln_id, invt_name_first, invt_name_middle, invt_name_last, 
        invt_seq_nr, is_US_resident, IFNULL(invt_country_code, 'NULL') AS invt_country_code
    FROM `usptobias.final_dataset.03_appln_inventors_uspto`

), gen_table AS(
    SELECT 
        name_first as invt_name_first, name_last AS invt_name_last, 
        IFNULL(country_code, 'NULL') AS invt_country_code, predicted_gender as invt_gender
    FROM `usptobias.final_dataset.15_name_gender` 
), origin_table AS(
    SELECT 
        name_first AS invt_name_first, name_last AS invt_name_last, country_code,
        country_origin, sub_region_origin
    FROM `usptobias.final_dataset.16_name_origin`
), ethnicity_table AS(
    SELECT name_first AS invt_name_first, name_last AS invt_name_last, ethnicity AS invt_ethnicity
    FROM `usptobias.final_dataset.14_name_ethnicity`
), country_gdp AS(
    SELECT country_code AS invt_country_code, gdppc AS invt_gdppc
    FROM `usptobias.data_preparation.6_gdppc_country`
    WHERE country_code IS NOT NULL
), invt_gen_table AS(
    SELECT *
    FROM invnt_table
    LEFT JOIN gen_table USING(invt_name_first, invt_name_last, invt_country_code)
    LEFT JOIN country_gdp USING(invt_country_code)
)

SELECT
    appln_id, invt_name_first, invt_name_middle, invt_name_last, invt_seq_nr, is_US_resident, invt_gdppc,
    (CASE WHEN invt_country_code<>'NULL' THEN invt_country_code ELSE NULL END) AS invt_country_code,
    invt_gender, country_origin As invt_country_origin, sub_region_origin as invt_sub_reg_origin, invt_ethnicity
FROM invt_gen_table
LEFT JOIN origin_table USING(invt_name_first, invt_name_last)
LEFT JOIN ethnicity_table USING(invt_name_first, invt_name_last)

"""

# Defining the query
query_job = client.query(query, location='US', job_config=job_config)
query_job.result()

In [28]:
client = bigquery.Client()
# Creating Job Config
job_config = bigquery.QueryJobConfig()
#job_config.dry_run = True
job_config.use_query_cache = False
# Set configuration.query.writeDisposition
job_config.write_disposition = 'WRITE_TRUNCATE'

# Set the destination table
dataset_id = 'final_dataset'
table_id = '17_examOrig_appln'
table_ref = client.dataset(dataset_id).table(table_id)
job_config.destination = table_ref

query= """
WITH examiners AS(
    SELECT examiner_id, examiner_name_first, examiner_name_middle, examiner_name_last, examiner_art_unit, 'US' AS country_code
    FROM `usptobias.final_dataset.04_examiners_uspto`
), gender AS(
    SELECT name_first AS examiner_name_first, name_last AS examiner_name_last, country_code, predicted_gender AS examiner_gender
    FROM `usptobias.final_dataset.15_name_gender`
), origin AS(
    SELECT 
        name_first AS examiner_name_first, name_last AS examiner_name_last, country_code,
        country_origin AS examiner_country_origin, 
        sub_region_origin AS examiner_sub_reg_origin
    FROM `usptobias.final_dataset.16_name_origin`
), ethnicity_table AS(
    SELECT name_first AS examiner_name_first, name_last AS examiner_name_last, ethnicity AS examiner_ethnicity
    FROM `usptobias.final_dataset.14_name_ethnicity`
), exm_gender AS(
    SELECT *
    FROM examiners
    LEFT JOIN gender USING(examiner_name_first, examiner_name_last, country_code)
)

SELECT 
    examiner_id,
    examiner_name_first, examiner_name_middle, examiner_name_last,
    examiner_art_unit,
    examiner_gender,
    examiner_country_origin,
    examiner_sub_reg_origin,
    examiner_ethnicity
FROM exm_gender
LEFT JOIN origin USING(examiner_name_first, examiner_name_last, country_code)
LEFT JOIN ethnicity_table USING(examiner_name_first, examiner_name_last)
"""

# Defining the query
query_job = client.query(query, location='US', job_config=job_config)
query_job.result()

In [30]:
english_speaking_countries = ['AG', 'AU', 'BS', 'BB', 'BZ', 'CA', 'DM', 'GD', 'GY', 'IE', 'JM', 'NZ', 'KN', 'LC', 'VC', 'TT', 'UK', 'GB', 'US', #From gov.uk
                              'KE', 'MT', 'NA', 'SG', 'GH', #from Wikipedia
                              'IN', 'NG', 'PK', 'RW', 'ZA', 'UG', 'ZW'] #  from Wikipedia

In [33]:
client = bigquery.Client()
# Creating Job Config
job_config = bigquery.QueryJobConfig()
#job_config.dry_run = True
job_config.use_query_cache = False
# Set configuration.query.writeDisposition
job_config.write_disposition = 'WRITE_TRUNCATE'

query_params = [
    bigquery.ArrayQueryParameter('english_speaking_countries', 'STRING', english_speaking_countries)
]

job_config.query_parameters = query_params

# Set the destination table
dataset_id = 'final_dataset'
table_id = '17_appln_examInvtOrigin'
table_ref = client.dataset(dataset_id).table(table_id)
job_config.destination = table_ref

## Note: Here I used MAX() for getting the numbers, since for any CASE where the first two conditions are not met, 
##...then we will get -1 (if that auth exists in docdb_family).
query= """
WITH appln_t AS(
    SELECT appln_id_1 AS appln_id, ARRAY_AGG(appln_id_2) AS twins_array
    FROM `usptobias.final_dataset.07_twin_appln`
    GROUP BY appln_id
), invt_t AS(
    SELECT 
        appln_id, 
        ARRAY_AGG(invt_country_origin) AS invt_country_array, 
        ARRAY_AGG(invt_sub_reg_origin) AS invt_region_array,
        (CASE WHEN AVG(CASE WHEN invt_country_code IN('UK', 'GB',  'US', 'CA', 'NZ', 'AU', 'IE') THEN 1 ELSE 0 END)>0.5 THEN 1 
         ELSE 0 END) AS invt_angSax,
        (CASE WHEN AVG(CASE WHEN invt_country_code IN UNNEST(@english_speaking_countries) THEN 1 ELSE 0 END)>0.5 THEN 1
         ELSE 0 END) AS invt_eng,
        (CASE WHEN AVG(CASE WHEN invt_country_code IN('US') THEN 0 ELSE 1 END)>0 THEN 1
         ELSE 0 END) AS invt_foreign,
        MAX(CASE WHEN invt_country_code IN('US') THEN 0 ELSE 1 END) AS invt_foreign_exist,
        (CASE WHEN AVG(CASE WHEN invt_gender='male' THEN 0 WHEN invt_gender='female' THEN 1 END)>0.5 THEN 1
         ELSE 0 END) AS invt_female,
        (CASE WHEN AVG(CASE WHEN invt_ethnicity='Muslim' THEN 1 ELSE 0 END)>0.5 THEN 1
         ELSE 0 END) AS invt_muslim,
        #MAX(CASE WHEN invt_ethnicity='Muslim' THEN 1 ELSE 0 END) AS invt_muslim,
        (CASE WHEN AVG(CASE WHEN invt_country_origin IN('CN')  THEN 1 ELSE 0 END)>0.5 THEN 1
         ELSE 0 END) AS invt_chinese,
         #MAX(CASE WHEN invt_country_origin='CN' THEN 1 ELSE 0 END) AS invt_chinese,
        (CASE WHEN AVG(CASE WHEN invt_country_origin IN('JP')  THEN 1 ELSE 0 END)>0.5 THEN 1
         ELSE 0 END) AS invt_japanese,
        MAX(invt_gdppc) AS invt_gdppc,
        (CASE WHEN AVG(CASE WHEN invt_ethnicity='EastAsian' THEN 1 ELSE 0 END)>0.5 THEN 1
         ELSE 0 END) AS invt_eastasian,
        (CASE WHEN AVG(CASE WHEN invt_country_code IN('CN')  THEN 1 ELSE 0 END)>0.5 THEN 1
         ELSE 0 END) AS invt_res_china
    FROM `usptobias.final_dataset.17_invtOrig_appln`
    GROUP BY appln_id
), t1 AS(
    SELECT 
        examiner_id, examiner_country_origin, examiner_sub_reg_origin, examiner_ethnicity, 
        (CASE WHEN examiner_gender='male' THEN 0 WHEN examiner_gender='female' THEN 1 END) AS examiner_female
    FROM `usptobias.final_dataset.17_examOrig_appln`
), exam_t AS(
    SELECT 
        appln_id, examiner_id, examiner_country_origin, examiner_sub_reg_origin, examiner_female,
        (CASE WHEN examiner_ethnicity='Muslim' THEN 1 ELSE 0 END) AS examiner_muslim,
        (CASE WHEN examiner_country_origin IN('CN')  THEN 1 ELSE 0 END) AS examiner_chinese,
        (CASE WHEN examiner_country_origin IN('JP')  THEN 1 ELSE 0 END) AS examiner_japanese,
        (CASE WHEN examiner_country_origin IN('US')  THEN 1 ELSE 0 END) AS examiner_US
    FROM(
        SELECT appln_id, examiner_id
        FROM `usptobias.final_dataset.05_applnInfo_uspto`
    ) AS a
    LEFT JOIN t1 USING(examiner_id)
), assignee_t AS(
    SELECT 
        appln_id, 
        (CASE WHEN AVG(CASE WHEN assignee_country_code IN('CN') THEN 1 ELSE 0 END)>0.5 THEN 1
         ELSE 0 END) AS assignee_china
    FROM `usptobias.final_dataset.10_assignee_appln`
    GROUP BY appln_id
)

SELECT 
    appln_id,
    (CASE 
        WHEN ARRAY_LENGTH(invt_country_array)=0 THEN NULL
        WHEN examiner_country_origin IN((SELECT invt_country FROM UNNEST(invt_country_array) AS invt_country)) THEN 1 
     ELSE 0 END) AS same_country_origin,
    (CASE 
        WHEN ARRAY_LENGTH(invt_region_array)=0 THEN NULL
        WHEN examiner_sub_reg_origin IN((SELECT invt_region FROM UNNEST(invt_region_array) AS invt_region)) THEN 1 
     ELSE 0 END) AS same_reg_origin,
    examiner_muslim,
    examiner_chinese,
    examiner_japanese,
    examiner_female,
    examiner_US,
    examiner_country_origin,
    invt_angSax,
    invt_eng,
    invt_foreign,
    invt_foreign_exist,
    invt_muslim,
    invt_chinese,
    invt_japanese,
    invt_female,
    invt_gdppc,
    invt_eastasian,
    invt_res_china,
    assignee_china
FROM appln_t
LEFT JOIN invt_t USING(appln_id)
LEFT JOIN exam_t USING(appln_id)
LEFT JOIN assignee_t USING(appln_id)
"""

# Defining the query
query_job = client.query(query, location='US', job_config=job_config)

query_job.result()

# 14. Creating Patent Portfolio Size for Each Assignee

This table contains information on the number of patents that the assginee of each application has filed in the U.S. jurisdiction and within the last 5 years of the current application filing date.   

**Source:** For creating this table, we use `10_assignee_appln` from "final dataset" and `TLS201_APPLN` from the PATSTAT dataset.

In [ ]:
client = bigquery.Client()
# Creating Job Config
job_config = bigquery.QueryJobConfig()
#job_config.dry_run = True
job_config.use_query_cache = False
# Set configuration.query.writeDisposition
job_config.write_disposition = 'WRITE_TRUNCATE'

# Set the destination table
dataset_id = 'final_dataset'
table_id = '18_appln_portfolioSize'
table_ref = client.dataset(dataset_id).table(table_id)
job_config.destination = table_ref
query="""
WITH t1 AS(
    SELECT appln_id, assignee_id, appln_filing_date AS filing_date
    FROM `usptobias.final_dataset.10_assignee_appln`
    LEFT JOIN (
        SELECT appln_id, appln_filing_date, appln_auth
        FROM  `usptobias.patstat.tls201`
    ) USING(appln_id)
    WHERE appln_auth='US'
), t2 AS(
    SELECT assignee_id, ARRAY_AGG(STRUCT(appln_id, filing_date)) AS filing_arr
    FROM t1
    GROUP BY assignee_id
), t3 AS(
    SELECT
        t1.appln_id,
        t2.assignee_id,
        t1.filing_date AS filing_date_curr,
        t2.filing_arr
    FROM t1
    INNER JOIN t2 ON t1.assignee_id=t2.assignee_id
)

SELECT 
    t3.appln_id, 
    COUNT(CASE WHEN (filing_date_curr>f_list.filing_date) AND 
                    (DATE_SUB(filing_date_curr, INTERVAL 5 YEAR)<f_list.filing_date) 
               THEN 1 END) AS portfolio_size
FROM t3, UNNEST(t3.filing_arr) AS f_list
GROUP BY t3.appln_id

"""

# Running the query
query_job = client.query(query, location='US', job_config=job_config)
query_job.result()

# 15. Creating Grant Outcome Information Table

This table contains the granting outcome data of the '8' jurisdictions and also the information of the average granting rate for the twins in these jurisdictions.
These eight jurisdictions are 'EP', 'JP', 'CN', 'KR', 'DE', 'CA', 'AU', 'TW'.

**Source:** For creating this table, we use `07_twin_appln` table from "final dataset" and `TLS201_APPLN` from the PATSTAT dataset.

In [ ]:
client = bigquery.Client()
# Creating Job Config
job_config = bigquery.QueryJobConfig()
#job_config.dry_run = True
job_config.use_query_cache = False
# Set configuration.query.writeDisposition
job_config.write_disposition = 'WRITE_TRUNCATE'

# Set the destination table
dataset_id = 'final_dataset'
table_id = '19_appln_grantInfo'
table_ref = client.dataset(dataset_id).table(table_id)
job_config.destination = table_ref

## Note: Here I used MAX() for getting the numbers, since for any CASE where the first two conditions are not met, 
##...then we will get -1 (if that auth exists in docdb_family).
query= """
WITH t0 AS(
    SELECT appln_id, docdb_family_id
    FROM `usptobias.patstat.tls201` AS a
    WHERE appln_auth='US'
), t1 AS(
    SELECT 
        a.appln_id,
        ARRAY_AGG(b.appln_auth) AS docdb_auth_array,
        AVG(CAST(CAST(b.granted AS INT64) AS FLOAT64)) AS docdb_avg_grant,
        COUNT(DISTINCT b.appln_auth) AS nb_docdb_appln_auth
    FROM(####
        SELECT appln_id, docdb_family_id
        FROM `usptobias.patstat.tls201` AS a
        WHERE appln_auth='US'
    ) AS a
    LEFT JOIN (
        SELECT *
        FROM `usptobias.patstat.tls201`
        WHERE appln_auth IN('EP', 'JP', 'CN', 'KR','DE', 'CA', 'AU', 'TW')
    ) AS b ON a.docdb_family_id=b.docdb_family_id
    GROUP BY a.appln_id
), t2 AS(
    SELECT *
    FROM `usptobias.final_dataset.07_twin_appln` 
    WHERE appln_auth_2 IN('EP', 'JP', 'CN', 'KR','DE', 'CA', 'AU', 'TW')
), t3 AS(
    SELECT 
        appln_id_1 AS appln_id, 
        AVG(CAST(CAST(granted_2 AS INT64) AS FLOAT64)) AS twins_avg_grant,
        COUNT(DISTINCT appln_auth_2) AS nb_twins_appln_auth
    FROM `usptobias.final_dataset.07_twin_appln` 
    GROUP BY appln_id_1
), t4 AS(
    SELECT 
        a.*,
        docdb_auth_array AS docdb_auth_array_US,
        docdb_avg_grant,
        nb_docdb_appln_auth,
        twins_avg_grant,
        nb_twins_appln_auth
    FROM t2 AS a
    LEFT JOIN t1 ON a.appln_id_1=t1.appln_id
    LEFT JOIN t3 ON a.appln_id_1=t3.appln_id
) 

SELECT 
    appln_id_1 AS appln_id_US,
    ANY_VALUE(family_id) AS family_id,
    ANY_VALUE(appln_filing_year_1) AS appln_filing_year_US,
    ANY_VALUE(CAST(granted_1 AS INT64)) AS granted_US,
    ANY_VALUE(docdb_avg_grant) AS docdb_avg_grant,
    ANY_VALUE(nb_docdb_appln_auth) AS nb_docdb_appln_auth,
    ANY_VALUE(twins_avg_grant) AS twins_avg_grant,
    ANY_VALUE(nb_twins_appln_auth) AS nb_twins_appln_auth,
    MAX(CASE 
        WHEN appln_auth_2='EP' AND granted_2=True THEN 1
        WHEN appln_auth_2='EP' AND granted_2=FALSE THEN 0
        WHEN (SELECT 'EP' IN UNNEST(docdb_auth_array_US))=TRUE THEN -1
    END) AS twin_EP,
    MAX(CASE 
        WHEN appln_auth_2='JP' AND granted_2=True THEN 1
        WHEN appln_auth_2='JP' AND granted_2=FALSE THEN 0
        WHEN (SELECT 'JP' IN UNNEST(docdb_auth_array_US))=TRUE THEN -1
    END) AS twin_JP,
    MAX(CASE 
        WHEN appln_auth_2='CN' AND granted_2=True THEN 1
        WHEN appln_auth_2='CN' AND granted_2=FALSE THEN 0
        WHEN (SELECT 'CN' IN UNNEST(docdb_auth_array_US))=TRUE THEN -1
    END) AS twin_CN,
    MAX(CASE 
        WHEN appln_auth_2='KR' AND granted_2=True THEN 1
        WHEN appln_auth_2='KR' AND granted_2=FALSE THEN 0
        WHEN (SELECT 'KR' IN UNNEST(docdb_auth_array_US))=TRUE THEN -1
    END) AS twin_KR,
    MAX(CASE 
        WHEN appln_auth_2='DE' AND granted_2=True THEN 1
        WHEN appln_auth_2='DE' AND granted_2=FALSE THEN 0
        WHEN (SELECT 'DE' IN UNNEST(docdb_auth_array_US))=TRUE THEN -1
    END) AS twin_DE,
    MAX(CASE 
        WHEN appln_auth_2='CA' AND granted_2=True THEN 1
        WHEN appln_auth_2='CA' AND granted_2=FALSE THEN 0
        WHEN (SELECT 'CA' IN UNNEST(docdb_auth_array_US))=TRUE THEN -1
    END) AS twin_CA,
    MAX(CASE 
        WHEN appln_auth_2='AU' AND granted_2=True THEN 1
        WHEN appln_auth_2='AU' AND granted_2=FALSE THEN 0
        WHEN (SELECT 'AU' IN UNNEST(docdb_auth_array_US))=TRUE THEN -1
    END) AS twin_AU,
    MAX(CASE 
        WHEN appln_auth_2='TW' AND granted_2=True THEN 1
        WHEN appln_auth_2='TW' AND granted_2=FALSE THEN 0
        WHEN (SELECT 'TW' IN UNNEST(docdb_auth_array_US))=TRUE THEN -1
    END) AS twin_TW
    
FROM t4
GROUP BY appln_id_1
"""

# Defining the query
query_job = client.query(query, location='US', job_config=job_config)

query_job.result()

# 16. Creating the Final Table For the Analysis

This table contains the final key variables from all the necessary tables in the "final_dataset" for applications with the filing year between 2002 and 2012 that are needed for conducting our analysis.

In [ ]:
client = bigquery.Client()
# Creating Job Config
job_config = bigquery.QueryJobConfig()
#job_config.dry_run = True
job_config.use_query_cache = False
# Set configuration.query.writeDisposition
job_config.write_disposition = 'WRITE_TRUNCATE'

# Set the destination table
dataset_id = 'final_dataset'
table_id = '20_final_table'
table_ref = client.dataset(dataset_id).table(table_id)
job_config.destination = table_ref

## Note: Here I used MAX() for getting the numbers, since for any CASE where the first two conditions are not met, 
##...then we will get -1 (if that auth exists in docdb_family).
query= """
WITH t1 AS(
    SELECT appln_id, attorney_id
    FROM `usptobias.final_dataset.01_applnId_attorney`
), t2 AS(
    SELECT *
    FROM `usptobias.final_dataset.02_applnId_publn`
), t3 AS(
    SELECT *
    FROM `usptobias.final_dataset.05_applnInfo_uspto`
), t4 AS(
    SELECT *
    FROM `usptobias.final_dataset.19_appln_grantInfo`
), t5 AS(
    SELECT *
    FROM `usptobias.final_dataset.17_appln_examInvtOrigin`
    WHERE examiner_muslim IS NOT NULL AND invt_eng IS NOT NULL
), t6 AS(
    SELECT appln_id, assignee_id
    FROM `usptobias.final_dataset.10_assignee_appln`
), t7 AS(
    SELECT assignee_id, attorney_id, grant_rate AS attorney_FE
    FROM `usptobias.final_dataset.11_attorneyPFE_on_assignee`
), t8 AS(
    SELECT assignee_id, examiner_id, grant_rate AS examiner_FE
    FROM `usptobias.final_dataset.13_examinerPFE_on_assignee`
), t9 AS(
    SELECT assignee_id, examiner_art_unit, grant_rate AS artunit_FE
    FROM `usptobias.final_dataset.12_artunitPFE_on_assignee`
), t10 AS(
    SELECT appln_id, portfolio_size
    FROM `usptobias.final_dataset.18_appln_portfolioSize`
)

SELECT 
    t4.*,
    t1.attorney_id AS attorney_id_US,
    t2.publn_claims_earliest AS publn_claims_earliest_US, t2.publn_claims_grant AS publn_claims_grant, 
    t2.publn_date_earliest AS publn_date_earliest_US, t2.publn_date_grant AS publn_date_grant_US, 
    t2.nb_applt AS nb_applt_US, t2.nb_invt AS nb_invt_US,
    t3.* EXCEPT(appln_id),
    t5.* EXCEPT(appln_id),
    t6.assignee_id,
    t7.attorney_FE,
    t8.examiner_FE,
    t9.artunit_FE,
    t10.portfolio_size
FROM t4
LEFT JOIN t1 ON t4.appln_id_US=t1.appln_id
LEFT JOIN t2 ON t4.appln_id_US=t2.appln_id
LEFT JOIN t3 ON t4.appln_id_US=t3.appln_id
LEFT JOIN t5 ON t4.appln_id_US=t5.appln_id
LEFT JOIN t6 ON t4.appln_id_US=t6.appln_id
LEFT JOIN t10 ON t4.appln_id_US=t10.appln_id
LEFT JOIN t7 ON t6.assignee_id=t7.assignee_id AND t1.attorney_id=t7.attorney_id
LEFT JOIN t8 ON t6.assignee_id=t8.assignee_id AND t3.examiner_id=t8.examiner_id
LEFT JOIN t9 ON t6.assignee_id=t9.assignee_id AND t3.examiner_art_unit=t9.examiner_art_unit
WHERE appln_filing_year_US BETWEEN 2002 AND 2012
"""

# Defining the query
query_job = client.query(query, location='US', job_config=job_config)

query_job.result()

## 16.2. Exporting the Final Table Into a CSV File

In [ ]:
### Exporting the Final Table
client = bigquery.Client()

# Set Source table
project_id = 'usptobias'
dataset_id = 'final_dataset'
table_id = '20_final_table'
table_ref = client.dataset(dataset_id, project=project_id).table(table_id)

# Set Destination
dest_bucket = 'uspto-data'
dest_folder = 'final_dataset'
dest_file_name = 'final_table.csv'
dest_uri = "gs://{0}/{1}/{2}".format(dest_bucket, dest_folder, dest_file_name)

extract_job = client.extract_table(table_ref, dest_uri, location='US')
print('Extract job has {} started!'.format(extract_job.job_id))
extract_job.result()
print('Job has finished and table {} has been exported to {} bucket!'.format(dest_file_name, dest_bucket))